# Restarting DMRG

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/block-hczhai/block2-preview/blob/master/docs/source/tutorial/restarting-dmrg.ipynb)

In [1]:
!pip install block2==0.5.3rc5 -qq --progress-bar off --extra-index-url=https://block-hczhai.github.io/block2-preview/pypi/
!pip install pyscf==2.3.0 -qq --progress-bar off

## Introduction

In this tutorial we explain how to restart a DMRG calculation using MPS stored in disk. When you use the ``pyblock2`` interface, the MPS data is automatically stored into the scratch folder so one can recover the calculation as long as the scratch data is not removed.


## Restarting from Interruption

One common case is that the DMRG calculation is interrupted by the ``Ctrl+C`` signal, or the job has reached the time limit. Let's say we have a normal DMRG, but interrupted in the middle of sweep 1 (counting from zero):

In [2]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

from pyscf import gto, scf

mol = gto.M(atom='C 0 0 0; C 0 0 1.2425', basis='ccpvdz', symmetry='d2h')
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=2, ncas=26, g2e_symm=8)

print("NCAS = %d NCASELEC = %d" % (ncas, n_elec))
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2,
                    stack_mem=4 << 30, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
ket = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)

bond_dims = [250] * 4 + [500] * 4 + [750] * 4 + [1000] * 4
noises = [1e-4] * 4 + [1e-5] * 12 + [0]
thrds = [1e-8] * 20

energy = driver.dmrg(mpo, ket, n_sweeps=28, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=2, twosite_to_onesite=20)
print('DMRG energy (variational) = %20.15f' % energy)

converged SCF energy = -75.3869023777059
NCAS = 26 NCASELEC = 8
Environment initialization | Nsites =    26 | Center =     0
 INIT-R <-- Site =   23 ..  Bmem =   200 B Rmem =   200 B T = 0.01
 INIT-R <-- Site =   22 ..  Bmem = 2.55 KB Rmem = 2.55 KB T = 0.00
 INIT-R <-- Site =   21 ..  Bmem = 22.6 KB Rmem = 22.6 KB T = 0.00
 INIT-R <-- Site =   20 ..  Bmem =  237 KB Rmem =  237 KB T = 0.03
 INIT-R <-- Site =   19 ..  Bmem = 3.46 MB Rmem = 1.62 MB T = 0.21
 INIT-R <-- Site =   18 ..  Bmem = 25.7 MB Rmem = 1.91 MB T = 0.32
 INIT-R <-- Site =   17 ..  Bmem = 31.7 MB Rmem = 2.39 MB T = 0.55
 INIT-R <-- Site =   16 ..  Bmem = 40.6 MB Rmem = 3.04 MB T = 0.86
 INIT-R <-- Site =   15 ..  Bmem = 51.7 MB Rmem = 3.84 MB T = 1.21
 INIT-R <-- Site =   14 ..  Bmem = 66.2 MB Rmem = 4.84 MB T = 1.77
 INIT-R <-- Site =   13 ..  Bmem = 83.1 MB Rmem = 6.77 MB T = 1.80
 INIT-R <-- Site =   12 ..  Bmem =  116 MB Rmem = 8.38 MB T = 4.38
 INIT-R <-- Site =   11 ..  Bmem =  127 MB Rmem = 10.0 MB T = 4.09
 INI

KeyboardInterrupt: ignored

Then we can restart the calculation using the following script. Note that we change ``driver.get_random_mps`` to ``driver.load_mps`` with the same tag, so that data in the scratch folder ``./tmp`` with that tag will loaded. If we want to use the previous sweep schedule, we can keep the sweep schedule unchanged, and when calling ``driver.dmrg``, we use the ``sweep_start`` parameter to set the sweep index of the interrupted sweep (counting from zero), and the ``forward`` parameter to indicate the direction of the first restarting sweep. Then the DMRG will restart from the middle of the interrupted sweep.

For the MPS loaded from the interrupted sweep, ``driver.adjust_mps`` should not be used after ``driver.load_mps``.

In [3]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

from pyscf import gto, scf

mol = gto.M(atom='C 0 0 0; C 0 0 1.2425', basis='ccpvdz', symmetry='d2h')
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=2, ncas=26, g2e_symm=8)

print("NCAS = %d NCASELEC = %d" % (ncas, n_elec))
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2,
                    stack_mem=4 << 30, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
ket = driver.load_mps(tag="KET", nroots=1)

bond_dims = [250] * 4 + [500] * 4 + [750] * 4 + [1000] * 4
noises = [1e-4] * 4 + [1e-5] * 12 + [0]
thrds = [1e-8] * 20

sweep_start = 1
forward = sweep_start % 2 == 0

energy = driver.dmrg(mpo, ket, n_sweeps=28, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=2, twosite_to_onesite=20, forward=forward, sweep_start=sweep_start)
print('DMRG energy (variational) = %20.15f' % energy)

converged SCF energy = -75.3869023777058
NCAS = 26 NCASELEC = 8
Environment initialization | Nsites =    26 | Center =    12
 INIT-L --> Site =    1 ..  Bmem =   200 B Rmem =   200 B T = 0.00
 INIT-L --> Site =    2 ..  Bmem = 2.86 KB Rmem = 2.86 KB T = 0.00
 INIT-L --> Site =    3 ..  Bmem = 26.6 KB Rmem = 26.6 KB T = 0.00
 INIT-L --> Site =    4 ..  Bmem =  237 KB Rmem =  237 KB T = 0.02
 INIT-L --> Site =    5 ..  Bmem = 3.46 MB Rmem = 1.38 MB T = 0.07
 INIT-L --> Site =    6 ..  Bmem = 20.4 MB Rmem = 1.91 MB T = 0.13
 INIT-L --> Site =    7 ..  Bmem = 27.4 MB Rmem = 2.63 MB T = 0.14
 INIT-L --> Site =    8 ..  Bmem = 35.1 MB Rmem = 3.56 MB T = 0.16
 INIT-L --> Site =    9 ..  Bmem = 44.6 MB Rmem = 4.54 MB T = 0.18
 INIT-L --> Site =   10 ..  Bmem = 54.2 MB Rmem = 5.57 MB T = 0.19
 INIT-L --> Site =   11 ..  Bmem = 64.3 MB Rmem = 6.78 MB T = 0.21
 INIT-L --> Site =   12 ..  Bmem = 75.4 MB Rmem = 7.88 MB T = 0.26
 INIT-R <-- Site =   23 ..  Bmem =   200 B Rmem =   200 B T = 0.01
 INI

## Restarting from backup MPS

We can set the ``restart_dir`` in ``DMRGDriver`` to a different directory name than ``scratch``, so that after each sweep, ``block2`` will store a copy the MPS (int the left canonicalized or right canonicalized form) in the given ``restart_dir`` directory.

We can then restart using the data from this directory when the ``scratch`` folder is removed. To do this, we can copy the
data from ``restart_dir`` to the ``scratch`` for the new DMRG calculation, and use ``driver.load_mps`` to load the MPS.

Note that for this case, since after each sweep, the MPS is either left canonicalized or right canonicalized, the next sweep direction can be automatically determined without the need to set the ``forward`` parameter in ``driver.dmrg``.